# SageMaker Pipelines - Creating multi-tenant Pipelines (SaaS)

This notebook demonstrates utilizing SageMaker Pipelines in a software as a service (SaaS) model. 

We will create tenant specific Pipelines showcasing how only each tenant can start their own Pipeline. 

This notebook has been tested on SageMaker Studio using the `Data Science` kernel on an `ml.t3.medium`. 


Please make sure your execution role attached to your Studio user profile has `SageMakerFullAccess` policy attached. 

Please make sure your execution role attached to your Studio user profile has `iam:CreatePolicy, iam:CreateRole, iam:AttachRolePolicy and SageMakerFullAccess` permissions/policy attached.

---

## Set up

First we need to:
1. import the `boto3` and the SageMaker SDK (`sagemaker`).

We will use the SagMaker SDK to create the Pipeline and boto3 to start the Pipeline.
The SageMaker SDK is a high-level SDK that wraps boto3. 

1. import the two SageMaker Pipelines. 

`pipeline_app1.py` and `pipeline_app2.py` contain the definition for each Pipeline to be instantiated. In this case they are the same. Please feel free to customize the Pipeline(s) as you need.  




In [ ]:
import boto3
import sagemaker

import pipeline_app1
import pipeline_app2

from scripts import utils

role = sagemaker.get_execution_role()

sm = boto3.client('sagemaker')
region = boto3.session.Session().region_name

## Create per-tenant Pipeline
Now that we have imported the two Pipelines we can instantiate it per teneant and create the Pipelines in SageMaker.

The below function takes a [Pipeline](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#pipeline) object as input and creates/updates it in SageMaker.

In [3]:
def create_pipeline(pipeline, tags):
    
    definition = pipeline.definition()
    role = sagemaker.get_execution_role()

    upsert_response = pipeline.upsert(
                role_arn=role,
                tags = tags,         
    )

### Create `tenant{i}-app{i}` Pipeline
 We will create a Pipline for each tenant-app combination. In our case we will create 4 Pipelines:
 
```
tenant1-app1
tenant1-app2
tenant2-app1
tenant2-app2

```

In [4]:
tenant_ids = ["tenant1", "tenant2"]
apps = [(pipeline_app1, "app1"), (pipeline_app2, "app2")]

processing_instance_type="ml.m5.xlarge"
processing_instance_count=1
training_instance_type="ml.m5.xlarge" 
processing_instance_count=1
training_instance_count=1

tenant_pipleines = []

for tenant_id in tenant_ids:
    
    for app in apps:
        
        print(f"Creating Pipline for: {tenant_id} - {app[1]} Pipeline ")
        
        pipeline = app[0].get_pipeline(
            tenant_id=tenant_id,
            region=region,
            role=None,
            default_bucket=None,
            sagemaker_project_arn=None,
            pipeline_name=f"{tenant_id}-{app[1]}",
            model_package_group_name=f"{tenant_id}-{app[1]}-PipelineAppPackageGroup",
            base_job_prefix=f"{tenant_id}-{app[1]}",
           
        )
        
        tags=[
            {
                'Key': 'TenantID',
                'Value': tenant_id
            },
            {
                'Key': 'AppID',
                'Value': app[1]
            }
        ]
        
        tenant_pipleines.append(pipeline)
        
        create_pipeline(pipeline, tags)


Creating Pipline for: tenant1 - app1 Pipeline 


The input argument instance_type of function (sagemaker.image_uris.retrieve) is a pipeline variable (<class 'sagemaker.workflow.parameters.ParameterString'>), which is not allowed. The default_value of this Parameter object will be used to override it. Please make sure the default_value is valid.
The input argument instance_type of function (sagemaker.image_uris.retrieve) is a pipeline variable (<class 'sagemaker.workflow.parameters.ParameterString'>), which is not allowed. The default_value of this Parameter object will be used to override it. Please make sure the default_value is valid.
/opt/conda/lib/python3.7/site-packages/sagemaker/workflow/pipeline_context.py:261: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  UserWarning,
No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config
Popping out 'CertifyForMarketplace' from the pipeline definition

Creating Pipline for: tenant1 - app2 Pipeline 


INFO:sagemaker.image_uris:Defaulting to only available Python version: py3


Creating Pipline for: tenant2 - app1 Pipeline 


INFO:sagemaker.image_uris:Defaulting to only supported image scope: cpu.
INFO:sagemaker.image_uris:Defaulting to only available Python version: py3
INFO:sagemaker.image_uris:Defaulting to only supported image scope: cpu.


Creating Pipline for: tenant2 - app2 Pipeline 


INFO:sagemaker.image_uris:Defaulting to only available Python version: py3


In [5]:
print("Created Pipelines:\n" + '\n'.join(pipeline.name for pipeline in tenant_pipleines))


Created Pipelines:
tenant1-app1
tenant1-app2
tenant2-app1
tenant2-app2


#### Now that we have created the Pipelines we can move onto Starting them. See [01-DemoStartTenantPipelines.ipynb](./01-DemoStartTenantPipelines.ipynb)